In [ ]:
# Importing Libraries
from functions.patches_functions import *
from spectral import *
from functions.dataset_functions import *
from functions.augmentations import *
from models.mft_function import *

In [2]:
# Loading Electrolyzers-HSI
output_dir = "Path to Electrolyzers dataset/Electrolyzers-HSI"

HSI, RGB, GT = load_triplets(output_dir)
HSI = spectral_binning(HSI)


In [3]:
# Apply band-wise normalization with mask to all HSIs
normalized_HSI_bandwise = [normalize_hsi_vector_wise_with_mask(hsi, gt) for hsi, gt in zip(HSI, GT)]
#test = normalize_hsi_bandwise_with_mask(test, test_GT)

(78000, 180)
(78000, 180)
(78000, 180)
(78000, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(78000, 180)
(92500, 180)
(92500, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(78000, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(78000, 180)
(78000, 180)
(78000, 180)
(78000, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(92500, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(78000, 180)
(78000, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(92500, 180)
(92500, 180)


In [4]:
# Train Test split
train_indices = [
    2, 3, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 
    26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 
    44, 45, 47, 48, 49, 51, 52, 53, 54
]

test_indices = [
    0, 1, 4, 5, 6, 7, 8, 12, 41, 46, 50
]

In [5]:
# Train Data Generation
Training_cubes, Training_masks, Training_rgb = Generate_data(train_indices, normalized_HSI_bandwise, GT, RGB)
print(len(Training_cubes), Training_cubes[0].shape, Training_masks[0].shape, np.unique(Training_masks[0]))

44 (240, 325, 180) (240, 325) [0 1]


In [6]:
# Test Data Generation
test, test_GT, test_rgb = Generate_data(test_indices, normalized_HSI_bandwise, GT, RGB)
print(len(test),test[0].shape,test_GT[0].shape,np.unique(test_GT[0]))

11 (240, 325, 180) (240, 325) [0 1 2 3]


In [7]:
# Compute a stratification label for each cube based on its corresponding mask.
strat_labels = [get_strat_label(compute_foreground_ratio(mask))
                for mask in Training_masks]

# Use scikit-learn's train_test_split with stratification.
# Note: With 44 cubes, 10% corresponds to about 4 cubes.
indices = np.arange(len(Training_cubes))
train_idx, val_idx = train_test_split(indices, test_size=0.1,
                                      stratify=strat_labels, random_state=42)

# Create the training and validation sets.
Training_cubes2 = [Training_cubes[i] for i in train_idx]
Training_masks2  = [Training_masks[i] for i in train_idx]
training_rgb = [Training_rgb[i] for i in train_idx]

Validation_cubes = [Training_cubes[i] for i in val_idx]
Validation_masks  = [Training_masks[i] for i in val_idx]
validation_rgb = [Training_rgb[i] for i in val_idx]

# Optional: Print the number of cubes in each set and the stratification distribution.
print("Total cubes:", len(Training_cubes))
print("Training cubes:", len(Training_cubes2))
print("Validation cubes:", len(Validation_cubes))

from collections import Counter
print("Overall strat labels:", Counter(strat_labels))
print("Training strat labels:", Counter([strat_labels[i] for i in train_idx]))
print("Validation strat labels:", Counter([strat_labels[i] for i in val_idx]))

Total cubes: 44
Training cubes: 39
Validation cubes: 5
Overall strat labels: Counter({'medium': 34, 'low': 10})
Training strat labels: Counter({'medium': 30, 'low': 9})
Validation strat labels: Counter({'medium': 4, 'low': 1})


In [10]:
# Creating HSI-RGB patches  for MFT training data
patch_size = 9
training_patches, training_rgb, training_label = create_hsi_rgb_patches_with_labels(Training_cubes2, Training_masks2, training_rgb, patch_size=patch_size, fill_value_X = 0, ignore_classes=[0, 7, 8])
training_patches.shape, training_rgb.shape, training_label.shape, np.min(training_label), np.max(training_label)

((290043, 9, 9, 180), (290043, 9, 9, 3), (290043,), 1, 5)

In [11]:
np.unique(training_label), np.min(training_label), np.max(training_label)

(array([1, 2, 3, 4, 5], dtype=uint8), 1, 5)

In [12]:
# Creating HSI-RGB patches  for MFT validation data

validation_patches,validation_rgb, validation_label = create_hsi_rgb_patches_with_labels(Validation_cubes, Validation_masks, validation_rgb, patch_size=patch_size, fill_value_X = 0, ignore_classes=[0, 7, 8])
validation_patches.shape, validation_rgb.shape, validation_label.shape, np.min(validation_label), np.max(validation_label)

((41811, 9, 9, 180), (41811, 9, 9, 3), (41811,), 1, 5)

In [13]:
# training_label = training_label - 1
# np.unique(training_label), np.min(training_label), np.max(training_label)

In [14]:
# Creating HSI-RGB patches  for MFT testing data

testing_patches, testing_rgb, testing_label = create_hsi_rgb_patches_with_labels(test, test_GT,test_rgb, patch_size=patch_size, fill_value_X = 0, ignore_classes=[0])
testing_patches.shape, testing_rgb.shape, testing_label.shape

((92315, 9, 9, 180), (92315, 9, 9, 3), (92315,))

In [15]:
# testing_label = testing_label - 1
# np.unique(testing_label), np.min(testing_label), np.max(testing_label)

In [16]:
# Model's Input Data Modification/Tokenization (HSI)
training_patches_reshaped = training_patches.reshape(training_patches.shape[0], 81, 180)
validation_patches_reshaped = validation_patches.reshape(validation_patches.shape[0], 81, 180)
testing_patches_reshaped = testing_patches.reshape(testing_patches.shape[0], 81, 180)
training_patches_reshaped.shape, validation_patches_reshaped.shape, testing_patches_reshaped.shape

((290043, 81, 180), (41811, 81, 180), (92315, 81, 180))

In [17]:
# Model's Input Data Modification/Tokenization (RGB)
training_rgb_reshaped = training_rgb.reshape(training_rgb.shape[0], 81, 3)
validation_rgb_reshaped = validation_rgb.reshape(validation_rgb.shape[0], 81, 3)
testing_rgb_reshaped = testing_rgb.reshape(testing_rgb.shape[0], 81, 3)
training_rgb_reshaped.shape, validation_rgb_reshaped.shape, testing_rgb_reshaped.shape

((290043, 81, 3), (41811, 81, 3), (92315, 81, 3))

In [18]:
# Saving patches
np.save('MFT_patches/training_patches_vector_norm.npy', training_patches_reshaped)
np.save('MFT_patches/training_rgb_patches.npy', training_rgb_reshaped)
np.save('MFT_patches/training_labels.npy', training_label)

np.save('MFT_patches/validation_patches_vector_norm.npy', validation_patches_reshaped)
np.save('MFT_patches/validation_rgb_patches.npy', validation_rgb_reshaped)
np.save('MFT_patches/validation_labels.npy', validation_label)

np.save('MFT_patches/testing_patches_vector_norm.npy', testing_patches_reshaped)
np.save('MFT_patches/testing_rgb_patches.npy', testing_rgb_reshaped)
np.save('MFT_patches/testing_labels.npy', testing_label)

In [19]:
# Check
x = np.load('MFT_patches/testing_patches_vector_norm.npy')
x.shape

(92315, 81, 180)